# 新コーディングマニュアル

https://contents.nii.ac.jp/korekara/about/sw_wg/pc202402


In [31]:
import fitz # import PyMuPDF
from pathlib import Path
import os
import re
import pandas as pd

# テキストを置換：項目名を▲▲で囲む
def text_replace(text):
   # text = re.sub(r'\n([^\d\.|^[A-Z])', r'\1', text)  # 不要な改行の削除
   # text = re.sub(r'( [A-Z][0-9])', r'\1', text)
   # text = re.sub(r'\n(\d{1,2}\.\d{1,2})([\.\d]*)([A-Z]*[^\)])', r'\n▲\1\2\3▲', text) # 〔
   text = re.sub(r'\n(\d{1,2}\.\d{1,2})([\.\d]*)(.+)〔', r'\n▲\1\2\3▲〔', text) # 
   text = re.sub(r'\n(\d{1,2}\.\d{1,2})([\.\d]*)([^\)])', r'\n▲\1\2\3▲', text)
   text = re.sub(r'^(\d{1,2}\.\d{1,2})([\.\d]*)([A-Z]*[^\)])', r'\n▲\1\2\3▲', text)
   # text = re.sub(r'^([\d\.]{2,3}[A-Z]*[^\)])', r'\n▲\1▲', text) # ▲0.1▲ 
   text = re.sub(r'\n([A-Z]\d[\.\d]*)', r'\n▲\1▲', text) # ▲B2.2▲  
   text = re.sub(r'→ \n▲(.+)▲', r'→\1', text) # (→ \n▲
   text = re.sub(r' \n▲(.+)▲ (から)\n▲(.+)▲', r'\1\2\3', text)
   text = re.sub(r'^(第.+?章)', r'▲\1▲', text)
   text = re.sub(r'▲(\d\d\d)▲', r'\1', text)
   text = re.sub(r'▲.\n▲(.+)▲', r'▲\1', text)
   
   # text = re.sub(r'▲([^A-B0-9\.].+)▲', r'\1', text)
   return text

# pymupdfでPDFをテキストに変換
def pdf2text(filepath):
   basename_without_ext = os.path.splitext(os.path.basename(filepath))[0]
   print(basename_without_ext)

   doc = fitz.open(filepath)  # open a supported document

   text = ""
   # text = chr(12).join([page.get_text() for page in doc])
   text = "".join([page.get_text() for page in doc])
   text = text_replace(text)
   return text, basename_without_ext

# テキストをDataFrameに変換
def text2df(text):
   # split()で文字列を分割して取得したリストにstrip()を適用する。
   # 空白を含むカンマ区切り文字列の余分な空白を除去してリスト内包表記でリスト化
   # list_text = [x.strip() for x in text[0].split('▲')]
   list_text = [x.strip() for x in text.split('▲')]
   list_text = [a for a in list_text if a != '']  # 内包表記で空の要素を駆逐する
   # print(list_text)

   # キーとバリューが交互に並ぶリストを辞書にする
   dict_text = dict(zip(list_text[0::2], list_text[1::2]))
   #print(dict_text)

   # 異なる長さのリストを含む辞書をpd.DataFrameに変換
   df2 = pd.DataFrame.from_dict(dict_text, orient='index')
   # df2
   df2.to_excel('pndas_to_excel.xlsx', sheet_name='new_sheet_name')
   return df2

"""
# PDFのファイル名を引数に、テキストファイルを返すとともにテキストファイルを保存
def pdf2text(filepath):
   # filepath = r"pdf/CM1_0.pdf"
   # text = pdf2text(filepath)
   basename_without_ext = os.path.splitext(os.path.basename(filepath))[0]
   print(basename_without_ext)  # filename
   # 戻り値の型はタプルとなる。text, basename_without_ext
   # path_w = r"cm0.txt"
   # other_filepath = os.path.join(os.path.dirname(filepath), text[1]) # basename_without_ext)
   #other_ext_filepath = other_filepath + '.txt'
   #print(other_ext_filepath)

   with open(other_ext_filepath, mode='w') as f:
      f.write(text[0])

   return text[0]
"""

# test
print(pdf2text("pdf/CM0.pdf")) # 戻り値の型はタプルとなる。text, basename_without_ext
test_dataframe = text2df(pdf2text("pdf/CM0.pdf")[0])
test_dataframe


CM0
("\n▲0.1 ▲データセット \n▲0.1A ▲〔通則〕 \n 総合目録データベースのデータセット構成は、大きくは図書と雑誌に分かれ、それぞれ\n書誌データセットと所蔵データセットが中心となっている。さらに、典拠コントロールを\n行うための著者名典拠データセット、著作典拠データセットがあり、これら全体で総合目\n録データベースを形成している。 \n 総合目録データベースの外周には参照データセットがある。参照データセットとは、外\n部機関作成データを目録システム用に変換したものである。 \n データセット構成及び各データセットの詳細については、「目録情報の基準 第6 版」\n（2.1 データセット構成）を参照のこと。 \n▲0.1.1 ▲図書と逐次刊行物 \n▲0.1.1A ▲〔通則〕 \n 当該資料が、図書であるか逐次刊行物であるかの区分は、原則として，資料の刊行方式\nによる。資料形態の種別は問わない。 \n詳細については、「目録情報の基準 第6 版」（2.2.1 図書と逐次刊行物）を参照のこと。 \n▲0.1.2 ▲和資料と洋資料 \n 総合目録データベースのデータセット上は、和資料と洋資料の区別はない。「日本目録\n規則2018 年版（NCR2018）」適用前は、準拠する目録規則が異なるため、本マニュアル\nでは、和資料と洋資料とを区別していた。NCR2018 適用後は、和資料洋資料ともに「日\n本目録規則2018 年版（NCR2018）」を適用するが、一部の細則では和資料洋資料で異な\nる扱いを行う箇所がある。また、データの記述に当たって適用する目録用言語は，原則と\nして日本語資料，中国語資料，韓国・朝鮮語資料については日本語，左記以外の資料につ\nいては英語とする。 \n なお、日本語および英語等のラテン文字以外で書かれた資料のうち、中国語、韓国・朝\n鮮語、アラビア文字、タイ文字、デーヴァナーガリー文字資料、それら以外全てに関する\n特殊文字・特殊言語資料については、別途定められている規則も参照すること。 \n▲0.1.2A▲〔通則〕 \n資料が和資料であるか洋資料であるかは、原則として、規定の情報源に表示されたタイ\nトルの言語による。\n 日本語、中国語、韓国・朝鮮語のタイトルを持つ資料(TTLL フィールドに記録されてい\nるコードが

,0
0.1,データセット
0.1A,〔通則〕 \n 総合目録データベースのデータセット構成は、大きくは図書と雑誌に分かれ、それぞ...
0.1.1,図書と逐次刊行物
0.1.1A,〔通則〕 \n 当該資料が、図書であるか逐次刊行物であるかの区分は、原則として，資料の刊行方...
0.1.2,和資料と洋資料 \n 総合目録データベースのデータセット上は、和資料と洋資料の区別はない。「...
...,...
B1.1.5,団体の名称において、組織の改編、再編、統廃合などに伴う名称の変更は、名称の相違\nと判断し、...
B1.1.6,B1.1.2 からB1.1.5 に従い、それぞれ典拠データを作成した場合は、各典拠データの間...
B1.1.7,次のような場合は、名称の相違とはしない。 \n \n (1) 字種・字体や綴りの相違 \...
B1.2,(識別要素) \n 識別要素の相違は、新規データ作成の根拠となりうる。 \n 例えば、次のよ...


In [32]:
# https://qiita.com/poorko/items/9140c75415d748633a10
# Pythonのスクレイピングで文字だけを抜き出す汎用的な方法
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse

# urlを指定してBeautifulSoupオブジェクトを返す
def scrape_html(url):
  html = requests.get( url ) # .text
  soup = BeautifulSoup(html.content.decode("utf-8", "ignore"), "html.parser")
  # print(soup.prettify)
  return soup

# BeautifulSoupオブジェクトを引数に、テキストを返す
def get_text(soup):
  # scriptやstyleを含む要素を削除(decompose)する  head と link<a>も抜く
  for script in soup(["script",  "style", "head", "a"]):
      script.decompose()
  # print(soup)
  # タグを削除してテキストを取得
  text = soup.get_text()
  # print(text)
  # textを改行ごとにリストに入れて、リスト内の要素の前後の空白を削除(strip)、内包表記でリスト化
  lines = [line.strip() for line in text.splitlines()]
  #lines = []
  # for line in text.splitlines():
  #  lines.append(line.strip())
  # print(lines)

  text = "\n".join(line for line in lines if line)
  # print(text)
  return text

# URLを引数に、リンクのリストを出力
def list_links(url):
    parse_html = scrape_html(url)
    title_lists = parse_html.find_all("a")

    # hrefを取得してリスト化
    name_list = []
    url_list = []
    for i in title_lists:
        name_list.append(i.string)
        url_list.append(i.get('href'))   # (i.attrs["href"]) getはなければNoneが返ってきます

    # リストの空の要素を消す
    # [x for x in url_list if x != '']
    url_list_b = [x for x in url_list if x is not None]
    # 外だし：先頭が数字のリンクを抽出し、リスト化
    # url_list_b = [x for x in url_list_b if re.match('^\d', x)]
    # print(url_list_b)
    
    return url_list_b

'''
# URLを引数にurlをパースして、リンクのリストを返す
def url_parse(access_url):
   parsed_url = urlparse(access_url) 
   path_list = parsed_url.path.split("/")[-2:]
   # フォーマットする 
   # url = '{uri.scheme}://{uri.netloc}/path_list[0]/'.format(uri=urlparse(access_url))
   url_d = parsed_url.scheme + '://' + parsed_url.netloc + '/' + path_list[0] + '/'
   print(url_d)

   # リンク先をリスト化
   #    url_list_b = list_links(access_url)
   # url_full = url_d + url_list_b[0]
   # htmlファイルをスクレイプする
   # print(scrape_html(url_d + url_list_b[0])) #.decode("utf-8")
   # html_text = get_text(scrape_html(url_full))

   # urlのリストから、url_fullをリスト化
   url_full = [url_d + i for i in url_list_b]
   """
   url_full = []
   for i in url_list_b:
      url_full.append(url_d + i)
      # html_text += get_text(scrape_html(url_full))
   """
   return url_full
'''

# テスト用
get_text(scrape_html("https://catill.bitbucket.io/CM/0_1.html")) ##https://qiita.com/poorko/items/9140c75415d748633a10"))
# list_links("https://catill.bitbucket.io/CM/mokuji.html")


'目録システムコーディングマニュアル（CAT2020対応版）\n[]\n[]改訂履歴\n[]0.1.1 図書と逐次刊行物\n0.1 データセット\n0.1A 〔通則〕\n総合目録データベースのデータセット構成は、大きくは図書と雑誌に分かれ、それぞれ書誌データセットと所蔵データセットが中心となっている。さらに、典拠コントロールを行うための著者名典拠データセット、統一書名典拠データセットがあり、これら全体で総合目録データベースを形成している。\n総合目録データベースの外周には参照データセットがある。参照データセットとは、外部機関作成データを目録システム用に変換したものである。\nデータセット構成及び各データセットの詳細については、「目録情報の基準 第5版」（2.1 データセット構成）を参照のこと。\n[]'

In [33]:
#コーディングマニュアルおよび目録情報の基準パブリックコメント
# https://contents.nii.ac.jp/korekara/about/sw_wg/pc202402
# リンク先ファイル名リストから、
pubcom_access_url = "https://contents.nii.ac.jp/korekara/about/sw_wg/pc202402"
pubcom_url_filename =   list_links(pubcom_access_url) # url_parse(access_url)
print(pubcom_url_filename)
# CM\d.+\.pdfのリンクを抽出し、リスト化
pubcom_url_filename_cm = [x for x in pubcom_url_filename if re.match('.*CM.\d+\.pdf$', x)]
print(pubcom_url_filename_cm)


['#main-content', '/korekara', '/korekara/about/sw_wg/pc202402', '/en/korekara/about/sw_wg/pc202402', '/korekara', '/korekara/about', '/korekara/about/sw_wg', '/korekara', '/korekara/news', '/korekara/about', '/korekara/about', '/korekara/about/sw_wg', '/korekara/about/sw_wg/member', '/korekara/about/sw_wg/history', '/korekara/about/sw_wg/documents', '/korekara/about/sw_wg/pc202312', '/korekara/about/sw_wg/pc202402', '/korekara/about/ug_wg', '/korekara/about/pastwg', '/korekara/rule', '/korekara/documents', '/korekara/libsysnw', '/korekara/contact', '/korekara/about/sw_wg/pc202312', '#purpose', '#period', '#form', '#cm-draft', '#kijun-draft', '#saisoku-draft', '#documents', 'https://contents.nii.ac.jp/sites/default/files/korekara/2024-02/CM0.pdf', 'https://contents.nii.ac.jp/sites/default/files/korekara/2024-02/CM1_0.pdf', 'https://contents.nii.ac.jp/sites/default/files/korekara/2024-02/CM1_1.pdf', 'https://contents.nii.ac.jp/sites/default/files/korekara/2024-02/CM_2_0.pdf', 'https://c

In [34]:
# PDFのダウンロード
import os
import urllib.request

def download_file(url, dst_path):
    with urllib.request.urlopen(url) as web_file, open(dst_path, 'wb') as local_file:
        local_file.write(web_file.read())

def download_file_to_dir(url, dst_dir):
    download_file(url, os.path.join(dst_dir, os.path.basename(url)))

dst_dir = 'pdf'
for i2 in pubcom_url_filename_cm:
    # download_file_to_dir(pubcom_url_filename_cm[1], dst_dir)
    download_file_to_dir(i2, dst_dir)

'''
full_text_cm = ''
#xxxxxfull_text_cm = get_text(scrape_html(pubcom_url_filename_cm[0]))
# for i in pubcom_url_filename_cm:
#     full_text_cm += get_text(scrape_html(i))
filepath = r"pdf/CM1_0.pdf"
text = pdf2text(filepath)
# 戻り値の型はタプルとなる。text, basename_without_ext
dataframe = text2df(text[0])
dataframe
'''


'\nfull_text_cm = \'\'\n#xxxxxfull_text_cm = get_text(scrape_html(pubcom_url_filename_cm[0]))\n# for i in pubcom_url_filename_cm:\n#     full_text_cm += get_text(scrape_html(i))\nfilepath = r"pdf/CM1_0.pdf"\ntext = pdf2text(filepath)\n# 戻り値の型はタプルとなる。text, basename_without_ext\ndataframe = text2df(text[0])\ndataframe\n'

In [35]:
import glob
files = glob.glob("pdf/*.pdf")
print(files)

['pdf\\CM0.pdf', 'pdf\\CM14.pdf', 'pdf\\CM1_0.pdf', 'pdf\\CM1_1.pdf', 'pdf\\CM6_0.pdf', 'pdf\\CM6_3.pdf', 'pdf\\CM6_4.pdf', 'pdf\\CM6_5.pdf', 'pdf\\CM8_1.pdf', 'pdf\\CM8_2.pdf', 'pdf\\CM8_3.pdf', 'pdf\\CM8_4.pdf', 'pdf\\CM_2_0.pdf', 'pdf\\CM_2_2.pdf', 'pdf\\CM_2_3.pdf', 'pdf\\CM_2_4.pdf']


In [36]:
#
pdf_text = ''
for i in files:
    pdf_text += pdf2text(i)[0] # 戻り値の型はタプルとなる。text, basename_without_ext

path_w = 'data/new_cm.txt'
with open(path_w, mode='w') as f:
    f.write(pdf_text)


CM0
CM14
CM1_0
CM1_1
CM6_0
CM6_3
CM6_4
CM6_5
CM8_1
CM8_2
CM8_3
CM8_4
CM_2_0
CM_2_2
CM_2_3
CM_2_4


In [37]:
newcm_dataframe = text2df(pdf_text)
newcm_dataframe

newcm_dataframe.to_excel('data/new_cm.xlsx',
            index=True, header=True)

# 旧（現行）CM

In [15]:
# リンク先ファイル名リストから、
former_access_url = "https://catill.bitbucket.io/CM/mokuji.html"
url_filename =   list_links(former_access_url) # url_parse(access_url)
# 外だし：先頭が数字のリンクを抽出し、リスト化
url_filename_digit = [x for x in url_filename if re.match('^\d', x)]
# print(url_filename_digit)

# urlの補完
def url_complement(s):
  parsed_url = urlparse(former_access_url)  # urlをパースする
  path_list = parsed_url.path.split("/")[-2:]
  # フォーマットする 
  # url_d = parsed_url.scheme + '://' + parsed_url.netloc + '/' + path_list[0] + '/'
  return parsed_url.scheme + '://' + parsed_url.netloc + '/' + path_list[0] + '/' + s  #該当する文字列を置換する

# map -リストの各要素に同じ処理を施す
url_full = list(map(url_complement, url_filename_digit)) #map関数でurl_filename_digitの各要素にurl_complementする
print(url_full)


['https://catill.bitbucket.io/CM/0_1.html', 'https://catill.bitbucket.io/CM/0_1_1.html', 'https://catill.bitbucket.io/CM/0_1_2.html', 'https://catill.bitbucket.io/CM/0_4.html', 'https://catill.bitbucket.io/CM/0_4_1.html', 'https://catill.bitbucket.io/CM/0_4_2.html', 'https://catill.bitbucket.io/CM/0_4_3.html', 'https://catill.bitbucket.io/CM/0_4_4.html', 'https://catill.bitbucket.io/CM/1.html', 'https://catill.bitbucket.io/CM/1_0_1.html', 'https://catill.bitbucket.io/CM/1_0_2.html', 'https://catill.bitbucket.io/CM/1_1_1.html', 'https://catill.bitbucket.io/CM/1_1_2.html', 'https://catill.bitbucket.io/CM/1_1_3.html', 'https://catill.bitbucket.io/CM/2.html', 'https://catill.bitbucket.io/CM/2_0.html', 'https://catill.bitbucket.io/CM/2_0_1.html', 'https://catill.bitbucket.io/CM/2_0_2.html', 'https://catill.bitbucket.io/CM/2_0_3.html', 'https://catill.bitbucket.io/CM/2_0_4.html', 'https://catill.bitbucket.io/CM/2_0_5.html', 'https://catill.bitbucket.io/CM/2_0_6.html', 'https://catill.bitbuck

In [16]:
print(url_full)
html_text = ""
for i in url_full:
    html_text += get_text(scrape_html(i))



['https://catill.bitbucket.io/CM/0_1.html', 'https://catill.bitbucket.io/CM/0_1_1.html', 'https://catill.bitbucket.io/CM/0_1_2.html', 'https://catill.bitbucket.io/CM/0_4.html', 'https://catill.bitbucket.io/CM/0_4_1.html', 'https://catill.bitbucket.io/CM/0_4_2.html', 'https://catill.bitbucket.io/CM/0_4_3.html', 'https://catill.bitbucket.io/CM/0_4_4.html', 'https://catill.bitbucket.io/CM/1.html', 'https://catill.bitbucket.io/CM/1_0_1.html', 'https://catill.bitbucket.io/CM/1_0_2.html', 'https://catill.bitbucket.io/CM/1_1_1.html', 'https://catill.bitbucket.io/CM/1_1_2.html', 'https://catill.bitbucket.io/CM/1_1_3.html', 'https://catill.bitbucket.io/CM/2.html', 'https://catill.bitbucket.io/CM/2_0.html', 'https://catill.bitbucket.io/CM/2_0_1.html', 'https://catill.bitbucket.io/CM/2_0_2.html', 'https://catill.bitbucket.io/CM/2_0_3.html', 'https://catill.bitbucket.io/CM/2_0_4.html', 'https://catill.bitbucket.io/CM/2_0_5.html', 'https://catill.bitbucket.io/CM/2_0_6.html', 'https://catill.bitbuck

In [17]:
path_w = 'text/cm_old.txt'

with open(path_w, mode='w') as f:
    f.write(html_text)

In [19]:
def text2df22(text):
   # split()で文字列を分割して取得したリストにstrip()を適用する。
   # 空白を含むカンマ区切り文字列の余分な空白を除去してリスト内包表記でリスト化
   # list_text = [x.strip() for x in text[0].split('▲')]
   list_text = [x.strip() for x in text.split('▲')]
   list_text = [a for a in list_text if a != '']  # 内包表記で空の要素を駆逐する
   print(list_text)

   # キーとバリューが交互に並ぶリストを辞書にする
   dict_text = dict(zip(list_text[0::2], list_text[1::2]))
   print(dict_text)

   # 異なる長さのリストを含む辞書をpd.DataFrameに変換
   df2 = pd.DataFrame.from_dict(dict_text, orient='index')
   df2
   df2.to_excel('pndas_to_excel.xlsx', sheet_name='new_sheet_name')
   return df2

# テキストを置換：項目名を▲▲で囲む
def text_replace22(text):

   # text = re.sub(r'\n([^\d\.|^[A-Z])', r'\1', text)  # 不要な改行の削除
   text = re.sub(r'目録システムコーディングマニュアル（CAT2020対応版）\n', r'', text)
   text = re.sub(r'\n([\d\.]{2,}[A-Z]*)', r'\n▲\1▲', text)
   text = re.sub(r'^([\d\.]{2,})', r'\n▲\1▲', text) # ▲0.1▲ 
   text = re.sub(r'([^A-Z][A-Z]\d[\.\d]*)', r'\n▲\1▲', text) # ▲B2.2▲  
   text = re.sub(r'→ \n▲(.+)▲', r'→\1', text) # (→ \n▲
   text = re.sub(r' \n▲(.+)▲ (から)\n▲(.+)▲', r'\1\2\3', text)
   text = re.sub(r'^(第.+?章)', r'\1▲', text)
   text = re.sub(r'▲(\d\d\d)▲', r'\1', text)
   text = re.sub(r'\[.+', r'', text)
   return text

former_dataframe = text2df22(text_replace22(html_text))


['0.1', 'データセット', '0.1A', '〔通則〕\n総合目録データベースのデータセット構成は、大きくは図書と雑誌に分かれ、それぞれ書誌データセットと所蔵データセットが中心となっている。さらに、典拠コントロールを行うための著者名典拠データセット、統一書名典拠データセットがあり、これら全体で総合目録データベースを形成している。\n総合目録データベースの外周には参照データセットがある。参照データセットとは、外部機関作成データを目録システム用に変換したものである。\nデータセット構成及び各データセットの詳細については、「目録情報の基準 第5版」（2.1 データセット構成）を参照のこと。', '0.1.1', '図書と逐次刊行物', '0.1.1A', '〔通則〕\n当該資料が、図書であるか逐次刊行物であるかの区分は、原則として，資料の刊行方式による。資料形態の種別は問わない。\n詳細については、「目録情報の基準 第5版」（2.2.1\u3000図書と逐次刊行物）を参照のこと。', '0.1.2', '和資料と洋資料\n総合目録データベースのデータセット上は、和資料と洋資料の区別はない。ただし、準拠する目録規則が異なるため、本マニュアルでは、和資料と洋資料とを区別している。\n目録対象資料を登録する場合、和資料ならば「日本目録規則1987年版改訂3版」を適用し、洋資料ならば「英米目録規則第2版(1988年改訂、1993年修正)」を適用し、書誌データセットに入力する。\nなお、日本語および英語等のラテン文字以外で書かれた資料のうち、中国語、韓国・朝鮮語、アラビア文字、タイ文字、デーヴァナーガリー文字資料、それら以外全てに関する特殊文字・特殊言語資料については、別途定められている規則も参照すること。', '0.1.2A', '〔通則〕\n資料が和資料であるか洋資料であるかは、原則として、規定の情報源に表示されたタイトルの言語による。\n日本語、中国語、韓国・朝鮮語のタイトルを持つ資料(TTLLフィールドに記録されているコードがjpn、chiまたはkor)については和資料として、また、日本語、中国語、韓国・朝鮮語以外のタイトルを持つ資料(TTLLフィールドに記録されているコードがjpn、chi、kor以外)については洋資料としてデータを作成する。\nただし、本文の内容

In [27]:
# print(pd.merge(newcm_dataframe, former_dataframe))
#newcm_dataframe.index
# former_dataframe.index
with pd.ExcelWriter('data/pandas_to_excel_multi.xlsx') as writer:
    former_dataframe.to_excel(writer, sheet_name='former')
    newcm_dataframe.to_excel(writer, sheet_name='new')
# former_dataframe.to_excel('data/former_cm.xlsx',
#            index=False, header=False)
# newcm_dataframe.to_excel('data/new_cm.xlsx',
#            index=False, header=False)
